## Failed Approach (Using API)

The following blocks try to use the API approach, which failed fantastically. I suggest you go to the next section, which works well.

You can also use directly PushishiftAPI() without psaw.

```Python
from pushshift_py import PushshiftAPI
import datetime as dt
import psaw
import pandas as pd
import requests
import json
import csv
import time
api = psaw.PushshiftAPI()

startEpoch = int(dt.datetime(2020,1,1).timestamp())
```
    
The following block shows how we can get information using pushshift. It shows how we can specify the features and get them. The returned data type is a generator with "submission" type as elements, though we can certainly make them into a list.

```Python
features = ['url','author', 'title', 'subreddit', 'id', 'created', 'score']
subreddit = 'NBA'

data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=10)

for datum in data:
    print(datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created, datum.score)

import praw

reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="AACAXZDE",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)
```

We can use praw to get the post body using the following block.
```Python
reddit.submission(id='eiev5d').selftext
```



In the following blocks, we create tables and store the information. For some reason, though, the api often acts up and freezes when we loop through the data.
```Python
import sqlite3

conn = sqlite3.connect('redditPosts.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Posts(
                id TEXT PRIMARY KEY,
                subreddit TEXT,
                title TEXT,
                author TEXT,
                url TEXT,
                created int)
                ''')

features = ['url','author', 'title', 'subreddit', 'id', 'created']
subreddit = 'stocks'
latest = dt.datetime(2021,5,8).timestamp()
earliest = dt.datetime(2020,1,1).timestamp()

startEpoch = earliest

while startEpoch <= latest:
    data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=100)
    
    for datum in data:
        print('Got here 2.')
        cur.execute('''INSERT OR IGNORE INTO Posts VALUES (?,?,?,?,?,?)'''
                    , (datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created))
        
        currentTime = datum.created
    
    conn.commit()
    if currentTime == startEpoch:
        break
    startEpoch = currentTime + 1
    print(dt.datetime.fromtimestamp(startEpoch))    

```

## Another Approach (Getting JSON)

The method above is shaky at best. A lot of times the api just freezes. On the other hand, I find using requests much easier. The following code blocks contain what you need for storing reddit data you need.

In [6]:
import requests
import datetime as dt
import sqlite3
import json
import time
import sys

In [2]:
def getPushShiftData(after,before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?size=100&after='+str(int(after))+'&before='+str(int(before))+'&subreddit='+str(sub)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

def extractInfo(datum,features):
    info = {}
    
    for feature in features:
        info[feature] = datum[feature]
    
    return info

def getLatestTime(data):
    return data[-1]['created_utc']

def dataStoragePipeline(after, before, sub, conn):
    features = ['full_link','author', 'title', 'subreddit', 'id', 'created_utc']
    cursor = conn.cursor()
    while after < before:
        data = getPushShiftData(after, before, sub)
        if not data:
            print("There is no data anymore.")
            return 1
        for datum in data:
            cursor.execute('''INSERT OR IGNORE INTO Posts 
                                VALUES (?,?,?,?,?,?)'''
                              , (datum['id'], datum['subreddit'], datum['title'], datum['author'], datum['full_link'], datum['created_utc']))
        
        after = getLatestTime(data) + 1
        conn.commit()
        print("The latest post is submitted at", dt.datetime.fromtimestamp(after-1))
        time.sleep(0.1)
    
        
        

In [2]:
conn = sqlite3.connect('redditPosts.sqlite')
cur = conn.cursor()
subreddit = 'dogecoin'
end = int(time.time()-86400) #I subtracted by one day, so that we have some buffer.
start = dt.datetime(2021,1,1).timestamp()
cur.execute('''SELECT MIN(created), MAX(created) FROM Posts
                WHERE subreddit = ?''', (subreddit,))
datatimes = cur.fetchone()

if datatimes[0]:
    dataEarly, dataLate = datatimes
    if end < dataEarly:
        end = dataEarly
    elif start < dataLate:
        start =dataLate

In [33]:
while start < end:
    try:
        flag = dataStoragePipeline(after = start, before = end, sub = subreddit, conn = conn)
        if flag:
            break
    except KeyboardInterrupt:
        print("Interrupted by keyboard. Stopping.")
        break
        
    except:
        print("Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.")
        time.sleep(1)
        cur.execute('''SELECT MIN(created), MAX(created) FROM Posts
                        WHERE subreddit = ?''', (subreddit,))
        datatimes = cur.fetchone()
        
        if datatimes[0]:
            dataEarly, dataLate = datatimes
            if end < dataEarly:
                end = dataEarly
            elif start < dataLate:
                start =dataLate
        

The latest post is submitted at 2021-01-01 18:30:03
The latest post is submitted at 2021-01-01 22:35:02
The latest post is submitted at 2021-01-02 00:02:23
The latest post is submitted at 2021-01-02 07:48:57
The latest post is submitted at 2021-01-02 09:50:20
The latest post is submitted at 2021-01-02 11:09:15
The latest post is submitted at 2021-01-02 12:10:58
The latest post is submitted at 2021-01-02 12:57:15
The latest post is submitted at 2021-01-02 13:42:57
The latest post is submitted at 2021-01-02 14:49:28
The latest post is submitted at 2021-01-02 16:44:42
The latest post is submitted at 2021-01-02 19:55:39
The latest post is submitted at 2021-01-02 22:45:04
The latest post is submitted at 2021-01-03 00:59:35
The latest post is submitted at 2021-01-03 06:49:31
The latest post is submitted at 2021-01-03 10:19:34
The latest post is submitted at 2021-01-03 12:28:51
The latest post is submitted at 2021-01-03 16:02:46
The latest post is submitted at 2021-01-03 19:43:45
The latest p

The latest post is submitted at 2021-01-28 17:30:12
The latest post is submitted at 2021-01-28 17:36:55
The latest post is submitted at 2021-01-28 17:39:54
The latest post is submitted at 2021-01-28 17:41:50
The latest post is submitted at 2021-01-28 17:44:45
The latest post is submitted at 2021-01-28 17:47:41
The latest post is submitted at 2021-01-28 17:50:05
The latest post is submitted at 2021-01-28 17:53:31
The latest post is submitted at 2021-01-28 17:57:04
The latest post is submitted at 2021-01-28 18:00:33
The latest post is submitted at 2021-01-28 18:03:07
The latest post is submitted at 2021-01-28 18:06:04
The latest post is submitted at 2021-01-28 18:09:30
The latest post is submitted at 2021-01-28 18:13:05
The latest post is submitted at 2021-01-28 18:16:46
The latest post is submitted at 2021-01-28 18:19:24
The latest post is submitted at 2021-01-28 18:22:35
The latest post is submitted at 2021-01-28 18:24:55
The latest post is submitted at 2021-01-28 18:26:48
The latest p

The latest post is submitted at 2021-01-28 22:31:54
The latest post is submitted at 2021-01-28 22:34:05
The latest post is submitted at 2021-01-28 22:35:48
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-28 22:37:54
The latest post is submitted at 2021-01-28 22:39:52
The latest post is submitted at 2021-01-28 22:41:57
The latest post is submitted at 2021-01-28 22:44:30
The latest post is submitted at 2021-01-28 22:46:49
The latest post is submitted at 2021-01-28 22:49:16
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-28 22:51:08
The latest post is submitted at 2021-01-28 22:53:02
The latest post is submitted at 2021-01-28 22:54:56
The latest post is submitted at 2021-01-28 22:57:17
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-28 22:59:20
The latest post i

The latest post is submitted at 2021-01-29 05:44:05
The latest post is submitted at 2021-01-29 05:49:11
The latest post is submitted at 2021-01-29 05:53:48
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 05:58:31
The latest post is submitted at 2021-01-29 06:02:40
The latest post is submitted at 2021-01-29 06:06:54
The latest post is submitted at 2021-01-29 06:10:54
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 06:16:04
The latest post is submitted at 2021-01-29 06:20:29
The latest post is submitted at 2021-01-29 06:24:14
The latest post is submitted at 2021-01-29 06:28:15
The latest post is submitted at 2021-01-29 06:31:22
The latest post is submitted at 2021-01-29 06:35:05
The latest post is submitted at 2021-01-29 06:37:48
The latest post is submitted at 2021-01-29 06:40:40
The latest post is submitted at 2021-01-29 06:43:4

The latest post is submitted at 2021-01-29 12:06:20
The latest post is submitted at 2021-01-29 12:08:40
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 12:11:13
The latest post is submitted at 2021-01-29 12:13:57
The latest post is submitted at 2021-01-29 12:17:12
The latest post is submitted at 2021-01-29 12:19:58
The latest post is submitted at 2021-01-29 12:22:35
The latest post is submitted at 2021-01-29 12:24:55
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 12:27:25
The latest post is submitted at 2021-01-29 12:30:05
The latest post is submitted at 2021-01-29 12:32:30
The latest post is submitted at 2021-01-29 12:34:42
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-01-29 12:36:48
The latest post is submitted at 2021-01-29 12:39:16
The latest post i

...

The latest post is submitted at 2021-05-05 13:53:12
The latest post is submitted at 2021-05-05 14:02:02
The latest post is submitted at 2021-05-05 14:10:39
The latest post is submitted at 2021-05-05 14:19:25
The latest post is submitted at 2021-05-05 14:29:53
The latest post is submitted at 2021-05-05 14:39:54
The latest post is submitted at 2021-05-05 14:50:03
The latest post is submitted at 2021-05-05 14:59:49
The latest post is submitted at 2021-05-05 15:09:52
The latest post is submitted at 2021-05-05 15:19:04
The latest post is submitted at 2021-05-05 15:28:30
The latest post is submitted at 2021-05-05 15:35:16
The latest post is submitted at 2021-05-05 15:43:13
The latest post is submitted at 2021-05-05 15:51:46
The latest post is submitted at 2021-05-05 16:00:43
The latest post is submitted at 2021-05-05 16:10:45
The latest post is submitted at 2021-05-05 16:20:41
The latest post is submitted at 2021-05-05 16:29:29
The latest post is submitted at 2021-05-05 16:38:59
The latest p

The latest post is submitted at 2021-05-06 21:00:44
The latest post is submitted at 2021-05-06 21:07:52
The latest post is submitted at 2021-05-06 21:12:57
The latest post is submitted at 2021-05-06 21:17:03
The latest post is submitted at 2021-05-06 21:24:05
The latest post is submitted at 2021-05-06 21:31:20
The latest post is submitted at 2021-05-06 21:37:53
The latest post is submitted at 2021-05-06 21:45:00
The latest post is submitted at 2021-05-06 21:52:04
The latest post is submitted at 2021-05-06 22:00:08
The latest post is submitted at 2021-05-06 22:07:50
The latest post is submitted at 2021-05-06 22:14:45
The latest post is submitted at 2021-05-06 22:22:38
The latest post is submitted at 2021-05-06 22:30:11
The latest post is submitted at 2021-05-06 22:37:48
The latest post is submitted at 2021-05-06 22:47:51
The latest post is submitted at 2021-05-06 22:58:38
The latest post is submitted at 2021-05-06 23:11:12
The latest post is submitted at 2021-05-06 23:25:45
The latest p

The latest post is submitted at 2021-05-08 00:21:51
The latest post is submitted at 2021-05-08 00:33:43
The latest post is submitted at 2021-05-08 00:45:32
The latest post is submitted at 2021-05-08 00:55:51
The latest post is submitted at 2021-05-08 01:07:29
The latest post is submitted at 2021-05-08 01:23:15
The latest post is submitted at 2021-05-08 01:36:28
The latest post is submitted at 2021-05-08 01:51:47
The latest post is submitted at 2021-05-08 02:08:08
The latest post is submitted at 2021-05-08 02:22:37
The latest post is submitted at 2021-05-08 02:40:13
The latest post is submitted at 2021-05-08 02:59:33
The latest post is submitted at 2021-05-08 03:20:02
The latest post is submitted at 2021-05-08 03:41:10
The latest post is submitted at 2021-05-08 04:05:26
The latest post is submitted at 2021-05-08 04:30:27
The latest post is submitted at 2021-05-08 04:52:13
The latest post is submitted at 2021-05-08 05:14:07
The latest post is submitted at 2021-05-08 05:29:52
The latest p

The latest post is submitted at 2021-05-08 22:29:21
The latest post is submitted at 2021-05-08 22:31:35
The latest post is submitted at 2021-05-08 22:33:39
The latest post is submitted at 2021-05-08 22:35:28
The latest post is submitted at 2021-05-08 22:36:48
The latest post is submitted at 2021-05-08 22:38:29
The latest post is submitted at 2021-05-08 22:40:22
The latest post is submitted at 2021-05-08 22:43:51
The latest post is submitted at 2021-05-08 22:44:57
The latest post is submitted at 2021-05-08 22:45:37
The latest post is submitted at 2021-05-08 22:46:34
The latest post is submitted at 2021-05-08 22:47:27
The latest post is submitted at 2021-05-08 22:48:59
The latest post is submitted at 2021-05-08 22:51:04
The latest post is submitted at 2021-05-08 22:52:46
The latest post is submitted at 2021-05-08 22:53:51
The latest post is submitted at 2021-05-08 22:54:59
The latest post is submitted at 2021-05-08 22:55:50
The latest post is submitted at 2021-05-08 22:56:42
The latest p

The latest post is submitted at 2021-05-09 06:36:20
The latest post is submitted at 2021-05-09 06:42:20
The latest post is submitted at 2021-05-09 06:48:27
The latest post is submitted at 2021-05-09 06:55:07
The latest post is submitted at 2021-05-09 06:59:50
The latest post is submitted at 2021-05-09 07:05:08
The latest post is submitted at 2021-05-09 07:08:12
The latest post is submitted at 2021-05-09 07:11:02
The latest post is submitted at 2021-05-09 07:14:35
The latest post is submitted at 2021-05-09 07:17:56
The latest post is submitted at 2021-05-09 07:21:22
The latest post is submitted at 2021-05-09 07:24:30
The latest post is submitted at 2021-05-09 07:28:03
The latest post is submitted at 2021-05-09 07:30:55
The latest post is submitted at 2021-05-09 07:34:37
The latest post is submitted at 2021-05-09 07:38:19
The latest post is submitted at 2021-05-09 07:42:03
The latest post is submitted at 2021-05-09 07:45:22
The latest post is submitted at 2021-05-09 07:48:36
The latest p

In [34]:
cur.execute('''SELECT subreddit,COUNT(*), COUNT(DISTINCT author) FROM Posts
                GROUP BY subreddit''')

print(cur.fetchall())

[('CryptoCurrency', 172461, 84948), ('GME', 145611, 40526), ('Superstonks', 146, 64), ('dogecoin', 576871, 169071), ('finance', 2941, 1267), ('options', 16589, 10251), ('pennystocks', 48514, 24938), ('stock', 100, 74), ('stocks', 23200, 13268), ('wallstreetbets', 954311, 499428)]


## Update the database by adding links.

In [4]:
cur.execute('''ALTER TABLE Posts
                ADD COLUMN ext_link TEXT''')


In [3]:
def dataUpdatePipeline(after, before, sub, conn):
    features = ['url', 'id']
    cursor = conn.cursor()
    while after < before:
        data = getPushShiftData(after, before, sub)
        if not data:
            print("There are no data anymore.")
            return 1
        for datum in data:
            cursor.execute('''UPDATE Posts
                                SET ext_link = ?
                                WHERE id = ? AND subreddit = ?'''
                              , (datum['url'], datum['id'], sub))
        
        after = getLatestTime(data) + 1
        conn.commit()
        print("The latest post is submitted at", dt.datetime.fromtimestamp(after-1))
        time.sleep(0.1)

def findStartingTime(cursor, subreddit):
    cursor.execute('''SELECT MAX(created) FROM Posts
                    WHERE subreddit = ? AND ext_link IS NOT NULL''', (subreddit,))
    datatimes = cursor.fetchone()
    
    return datatimes[0]

def getYourExistingSubs(cursor):
    cur.execute('''SELECT subreddit FROM Posts GROUP BY subreddit ORDER BY COUNT(*) ASC''')

    subreddits = list(map(lambda x: x[0], cur.fetchall()))
    
    return subreddits


In [46]:
#This chunk gives you all the subreddits you have scraped up to this point. Makes your life easier.
subreddits = getYourExistingSubs(cur)

In [50]:
for subreddit in subreddits:
    flag = 0
    print("Start working on {}".format(subreddit))
    end = int(time.time()-86400) #I subtracted by one day, so that we have some buffer.
    start = dt.datetime(2021,1,1).timestamp()

    dataTime = findStartingTime(cur,subreddit)
    if dataTime:
        start = max(start, dataTime)
    while True:
        try:
            dataUpdatePipeline(start, end, subreddit, conn)
            break
        except KeyboardInterrupt:
            print("Interrupted by keyboard. Stopping.")
            flag = 1
            break
        except:
            print("Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.")
            dataTime = findStartingTime(cur,subreddit)
            if dataTime:
                start = max(start, dataTime)
            time.sleep(1)
    
    if flag:
        break
            
            

Start working on Superstonks
There are no data anymore.
Start working on finance
The latest post is submitted at 2021-05-10 10:31:59
There are no data anymore.
Start working on options
The latest post is submitted at 2021-05-10 10:19:34
There are no data anymore.
Start working on pennystocks
The latest post is submitted at 2021-05-10 10:32:48
There are no data anymore.
Start working on GME
The latest post is submitted at 2021-05-09 19:10:47
The latest post is submitted at 2021-05-10 00:54:21
The latest post is submitted at 2021-05-10 07:37:07
The latest post is submitted at 2021-05-10 09:46:56
The latest post is submitted at 2021-05-10 10:36:15
There are no data anymore.
Start working on CryptoCurrency
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
The latest post is submitted at 2021-05-09 16:24:57
The latest post is submitted

The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
The latest post is submitted at 2021-05-09 16:24:57
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post i

The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
The latest post is submitted at 2021-05-09 16:24:57
The latest post is submitted at 2021-05-09 17:09:24
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post i

The latest post is submitted at 2021-05-09 14:32:26
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
The latest post is submitted at 2021-05-09 16:24:57
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021

The latest post is submitted at 2021-05-09 15:44:59
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-09 13:54:10
The latest post is submitted at 2021-05-09 14:32:26
The latest post is submitted at 2021-05-09 15:07:12
The latest post is submitted at 2021-05-09 15:44:59
The latest post is submitted at 2021-05-09 16:24:57
Error occurred. Probably due to frequent requests. Will resume workin

...

The latest post is submitted at 2021-03-27 21:31:52
The latest post is submitted at 2021-03-27 23:29:06
The latest post is submitted at 2021-03-28 02:36:34
The latest post is submitted at 2021-03-28 06:19:30
The latest post is submitted at 2021-03-28 09:42:34
The latest post is submitted at 2021-03-28 11:08:13
The latest post is submitted at 2021-03-28 12:15:29
The latest post is submitted at 2021-03-28 13:25:27
The latest post is submitted at 2021-03-28 14:42:05
The latest post is submitted at 2021-03-28 16:11:21
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-03-28 17:29:16
The latest post is submitted at 2021-03-28 19:01:40
The latest post is submitted at 2021-03-28 20:47:08
The latest post is submitted at 2021-03-28 22:39:39
The latest post is submitted at 2021-03-29 00:17:24
The latest post is submitted at 2021-03-29 02:05:37
The latest post is submitted at 2021-03-29 03:52:32
The latest post is submitted at

The latest post is submitted at 2021-04-05 11:25:34
The latest post is submitted at 2021-04-05 12:17:39
The latest post is submitted at 2021-04-05 13:34:41
The latest post is submitted at 2021-04-05 14:49:34
The latest post is submitted at 2021-04-05 15:58:07
The latest post is submitted at 2021-04-05 17:15:10
The latest post is submitted at 2021-04-05 18:52:01
The latest post is submitted at 2021-04-05 20:34:37
The latest post is submitted at 2021-04-05 23:01:30
The latest post is submitted at 2021-04-06 02:01:48
The latest post is submitted at 2021-04-06 05:43:55
The latest post is submitted at 2021-04-06 07:43:32
The latest post is submitted at 2021-04-06 08:47:55
The latest post is submitted at 2021-04-06 09:49:23
The latest post is submitted at 2021-04-06 10:42:16
The latest post is submitted at 2021-04-06 11:41:54
The latest post is submitted at 2021-04-06 12:43:28
The latest post is submitted at 2021-04-06 13:34:18
The latest post is submitted at 2021-04-06 14:31:54
The latest p

The latest post is submitted at 2021-04-16 20:58:15
The latest post is submitted at 2021-04-16 22:18:11
The latest post is submitted at 2021-04-16 23:59:48
The latest post is submitted at 2021-04-17 02:56:10
The latest post is submitted at 2021-04-17 06:58:36
The latest post is submitted at 2021-04-17 09:25:06
The latest post is submitted at 2021-04-17 10:59:50
The latest post is submitted at 2021-04-17 12:37:51
The latest post is submitted at 2021-04-17 14:09:26
The latest post is submitted at 2021-04-17 15:41:39
The latest post is submitted at 2021-04-17 17:12:32
The latest post is submitted at 2021-04-17 19:10:25
The latest post is submitted at 2021-04-17 20:59:20
The latest post is submitted at 2021-04-17 23:02:46
The latest post is submitted at 2021-04-18 02:10:24
The latest post is submitted at 2021-04-18 06:57:44
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-04-18 09:21:36
The latest post is submitted at

The latest post is submitted at 2021-04-26 18:33:16
The latest post is submitted at 2021-04-26 19:20:26
The latest post is submitted at 2021-04-26 20:26:41
The latest post is submitted at 2021-04-26 21:29:35
The latest post is submitted at 2021-04-26 22:34:41
The latest post is submitted at 2021-04-27 00:09:05
The latest post is submitted at 2021-04-27 01:51:00
The latest post is submitted at 2021-04-27 03:56:04
The latest post is submitted at 2021-04-27 05:32:40
The latest post is submitted at 2021-04-27 06:56:01
The latest post is submitted at 2021-04-27 07:54:15
The latest post is submitted at 2021-04-27 08:39:45
The latest post is submitted at 2021-04-27 09:08:50
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-04-27 09:37:55
The latest post is submitted at 2021-04-27 10:13:43
The latest post is submitted at 2021-04-27 10:52:00
The latest post is submitted at 2021-04-27 11:30:32
The latest post is submitted at

The latest post is submitted at 2021-05-05 11:16:44
The latest post is submitted at 2021-05-05 11:56:48
The latest post is submitted at 2021-05-05 12:49:11
The latest post is submitted at 2021-05-05 13:49:08
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-05 14:37:46
The latest post is submitted at 2021-05-05 15:30:22
The latest post is submitted at 2021-05-05 16:19:25
The latest post is submitted at 2021-05-05 17:07:39
The latest post is submitted at 2021-05-05 18:12:58
The latest post is submitted at 2021-05-05 19:19:51
Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.
The latest post is submitted at 2021-05-05 20:34:17
The latest post is submitted at 2021-05-05 21:36:08
The latest post is submitted at 2021-05-05 22:40:30
The latest post is submitted at 2021-05-05 23:35:10
The latest post is submitted at 2021-05-06 00:27:20
Error occurred. Probably due to frequent requests.

In [37]:
cur.execute('''SELECT subreddit FROM Posts GROUP BY subreddit ORDER BY COUNT(*) ASC''')

subreddits = list(map(lambda x: x[0], cur.fetchall()))

subreddits

['Superstonks',
 'finance',
 'options',
 'pennystocks',
 'GME',
 'CryptoCurrency',
 'dogecoin',
 'wallstreetbets']

# Using PRAW to Download Additional Features

In this section, we will use praw to scrap post body, score, and upvote_ratio.

**Please use your own client_id, secret, etc. since we might be scraping at the same time. I don't want to get this account locked up.**


In [1]:
import sqlite3
import praw
import time
import datetime as dt

conn = sqlite3.connect('redditPosts.sqlite')
cur = conn.cursor()

reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="AACAXZDE",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)


```Python
Sub = reddit.submission(id='eiev5d')
print(Sub.title, Sub.score, Sub.upvote_ratio)

#print(vars(Sub))

subreddit = 'options'
cur.execute('''SELECT id FROM Posts
                WHERE subreddit == ?
                LIMIT 10''', (subreddit,))

ids = cur.fetchall()


tStart = time.time()
for i,postId in enumerate(ids):
    Sub = reddit.submission(id=postId[0])
    print(Sub.selftext, Sub.score, Sub.upvote_ratio, Sub.url)
    

tEnd = time.time()

print(tEnd-tStart)


cur.execute('''CREATE TABLE IF NOT EXISTS PostBodyAndScore (
                    id TEXT PRIMARY KEY,
                    body TEXT,
                    score INT,
                    upvote_ratio FLOAT)''')

```

In [2]:
def timeLookupPRAW(cursor, subreddit):
    cursor.execute('''SELECT MIN(P.created), MAX(P.created)
                        FROM Posts P JOIN PostBodyAndScore B
                        ON P.id = B.id
                        WHERE subreddit = ?''', (subreddit,))
    return cursor.fetchone()

def retrieveID(cursor, num, subreddit, start, end):
    cursor.execute('''SELECT P.id
                        FROM (SELECT id, created FROM Posts
                                WHERE created >= ? AND created <= ? AND subreddit = ?) P LEFT JOIN PostBodyAndScore B
                        ON P.id = B.id
                        WHERE B.score IS NULL
                        ORDER BY P.created ASC
                        LIMIT ?''', (start,end, subreddit, num))
    
    return cursor.fetchall()

def storePostBodyAndScore(cursor, reddit, ids):
    flag = 0
    for i,postId in enumerate(ids):
        try:
            Sub = reddit.submission(id=postId[0])
            cursor.execute('''INSERT OR IGNORE INTO PostBodyAndScore
                                VALUES (?,?,?,?)''', (postId[0],Sub.selftext,Sub.score,Sub.upvote_ratio))
        except KeyboardInterrupt:
            flag = 1
            break
        except:
            print("Some error happened. Will abandon this post.")
            continue
        
    print("Finished adding post info for this batch.")
    conn.commit()
    return flag

def fillOutTable(cursor, reddit, subreddit, start, end, num = 500):
    delta = 86400
    
    _, latest = timeLookupPRAW(cursor,subreddit)
    
    if latest:
        start = max(start,latest)
    
    while start < end:
        ids = retrieveID(cursor, num, subreddit, start, start + delta)
        if not ids:
            start += delta
            print("==Finished all posts up to {}==".format(dt.datetime.fromtimestamp(start)))
            continue
        
        flag = storePostBodyAndScore(cursor, reddit, ids)
        if flag:
            return flag
            

    

In [8]:
#subreddits = getYourExistingSubs(cur)
subreddits = ['GME','wallstreetbets']
end = int(time.time()-86400)
start = dt.datetime(2021,1,1).timestamp()

for subreddit in subreddits:
    print("Currently working on subreddit {}".format(subreddit))
    flag = fillOutTable(cur, reddit, subreddit, start, end)
    if flag:
        break


Currently working on subreddit GME
==Finished all posts up to 2021-05-10 14:10:04==
==Finished all posts up to 2021-05-11 14:10:04==
==Finished all posts up to 2021-05-12 14:10:04==
==Finished all posts up to 2021-05-13 14:10:04==
Currently working on subreddit wallstreetbets
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.
==Finished all posts up to 2021-01-20 16:15:57==
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.
Finished adding post info for this batch.


## An alternative way of scraping PRAW

The way of requesting for information of a post by its id is just too slow. Maybe it is better to request posts of a certain time period and map the ids back. After one scan, we will then request by id.